# Exploring London's Transport API

During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

In [6]:
# Import all the relevant modules
import requests as re
from IPython.display import JSON

In [26]:
# Create a function to request data from API

def api_request(url, param={}):
    res = re.get(url=url, params=param)
    status_code = res.status_code
    
    if status_code != 200:
        print(f"Request error: {status_code}")
        return
    else:
        return res.json()

### Question 1: Obtain Air Quality Predictions for Tomorrow

Parse the dictionary and print the AirQuality predictions for tomorrow

In [53]:
#Get data from API
air_quality_endpoint = "https://api.tfl.gov.uk/AirQuality"
air_quality_data = api_request(air_quality_endpoint)

JSON(air_quality_data)

<IPython.core.display.JSON object>

In [54]:
#Get weather forecast, clean data
air_predictions_tomorrow = air_quality_data["currentForecast"][1]["forecastText"]
air_predictions_tomorrow = air_predictions_tomorrow.replace("&lt;br/&gt;", " ").replace("&#39;", "'")
print(air_predictions_tomorrow)

Friday will see the weather continuing to improve, with a dry, bright and sunny day.  A light breeze may cause some build-up of local emissions, overall air pollution is expected to remain 'Low' for the following pollutants:   Nitrogen Dioxide Ozone PM10 Particulate PM2.5 Particulate Sulphur Dioxide         


### Question 2: Obtain Transport of London's Modes of Transport

What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [52]:
# Get data from API
mode_endpoint = "https://api.tfl.gov.uk/Line/Meta/Modes"
mode_endpoint_data = api_request(mode_endpoint)

JSON(mode_endpoint_data)

<IPython.core.display.JSON object>

In [51]:
transport_modes = [mode['modeName'] for mode in mode_endpoint_data]
num_of_transport_modes = len(transport_modes)

print("The different transport modes in London are: ", transport_modes)
print("The number of transport modes in London are: ", num_of_transport_modes)

The different transport modes in London are:  ['bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'interchange-keep-sitting', 'interchange-secure', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']
The number of transport modes in London are:  18


### Question 3: How many bikepoints are operated by Transport of London? How many docks? 

How many BikePoints in London are operated by Transport for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [50]:
# Get data from API
bike_endpoint = "https://api.tfl.gov.uk/BikePoint/"
bike_data = api_request(bike_endpoint)

JSON(bike_data)

<IPython.core.display.JSON object>

In [49]:
# Length of list = number of bikepoint!
num_bikepoint = len(bike_data)
print(f"{num_bikepoint} bikepoints are operated by Transport of London")

790 bikepoints are operated by Transport of London


In [63]:
# How many docks are in all bikepoints?

dock_num = []
emptydock_num = []
bikes_num = []

for bikepoint in bike_data:
    dock_num.append(int(bikepoint['additionalProperties'][-1]['value']))
    emptydock_num.append(int(bikepoint['additionalProperties'][-2]['value']))
    bikes_num.append(int(bikepoint['additionalProperties'][-3]['value']))
sum(dock_num)

20990

### Question 4: How many tubes and bus lines are in London?

How many tube and bus lines are in London? Print names of all tube lines.

In [31]:
# How many tube and bus lines are in London? 

bus_endpoint = "https://api.tfl.gov.uk/Line/Mode/bus"
tube_endpoint = "https://api.tfl.gov.uk/Line/Mode/tube"

bus_data = api_request(bus_endpoint)
tube_data = api_request(tube_endpoint)

In [47]:
JSON(bus_data)

<IPython.core.display.JSON object>

In [48]:
JSON(tube_data)

<IPython.core.display.JSON object>

In [68]:
num_of_bus = len(bus_data)
num_of_tube = len(tube_data)
name_of_tube = [item["name"] for item in tube_data]

print(f"There are {num_of_bus} bus lines in London")
print(f"There are {num_of_tube} tube lines in London")
print("Name of tubes: ", name_of_tube)

There are 679 bus lines in London
There are 11 tube lines in London
Name of tubes:  ['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


### Question 5: Planning the Journey from London Heathrow to Tower Bridge

Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [61]:
# Must get latitude and longitude; tried with names of the sites, messed it up
ORIGIN = "51.470020, -0.454295"
DESTINATION = "51.505554, -0.075278"

journey_endpoint = f"https://api.tfl.gov.uk/Journey/JourneyResults/{ORIGIN}/to/{DESTINATION}"

bus_route_data = api_request(url=journey_endpoint, param={"mode":"bus"})
tube_route_data = api_request(url=journey_endpoint, param={"mode": "tube"})

In [45]:
JSON(bus_route_data)

<IPython.core.display.JSON object>

In [46]:
JSON(tube_route_data)

<IPython.core.display.JSON object>

In [66]:
bus_duration = [item['duration'] for item in bus_route_data["journeys"]]
tube_duration = [item['duration'] for item in tube_route_data["journeys"]]

print("Planned Duration: ") ## Find the fastest
print(f"Bus: {min(bus_duration)} minutes")
print(f"Tube: {min(tube_duration)} minutes")

Planned Duration: 
Bus: 132 minutes
Tube: 85 minutes


### Question 6: How many stations have the name "Victoria"?

How many station has `victoria` line?

In [41]:
station_url = "https://api.tfl.gov.uk/StopPoint/Search/"

station_data = api_request(url=station_url, param={"query": "victoria"})
JSON(station_data)

<IPython.core.display.JSON object>

In [43]:
num_victoria = len(station_data['matches'])
print(f"{num_victoria} stations have the name 'victoria'")

38 stations have the name 'victoria'
